In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import os

clean_data = pd.read_csv('/Users/ioanahritcu/Documents/Projects/boston housing/data/boston_clean.csv')

In [71]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(clean_data)

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_data, clean_data['MEDV'], test_size=0.25, random_state=33)


In [73]:

l_regression = LinearRegression()
trained_model = l_regression.fit(X_train, y_train)
y_train_predict = l_regression.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
r2 = r2_score(y_train, y_train_predict)




In [74]:
y_train_predict = l_regression.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
r2 = r2_score(y_train, y_train_predict)
print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print('MAE score is {}'.format(mean_absolute_error(y_train, y_train_predict)))
# model evaluation for testing set


The model performance for training set
--------------------------------------
RMSE is 1.8210269358786414e-14
R2 score is 1.0
MAE score is 1.4612592093050016e-14


In [75]:
y_test_predict = l_regression.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
r2 = r2_score(y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print('MAE scors is {}'.format(mean_absolute_error(y_test, y_test_predict)))

The model performance for testing set
--------------------------------------
RMSE is 1.9108636347594685e-14
R2 score is 1.0
MAE scors is 1.550340704956235e-14


In [76]:
# retrain with all data and save model
all_data = scaler.fit_transform(clean_data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']])

In [77]:

all_data_model = l_regression.fit(all_data, clean_data['MEDV'])
all_data_model_predict = l_regression.predict(all_data)


In [78]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()           
drive = GoogleDrive(gauth) 

In [79]:
# #upload files to google drive

# upload_file_list = ['metrics.txt']
# for upload_file in upload_file_list:
# 	gfile = drive.CreateFile({'parents': [{'id': '1qgOzoq9yZX8uq93s51q5cPLjBzeZhfrK'}]})
# 	# Read file and set it as the content of this instance.
# 	gfile.SetContentFile(upload_file)
# 	gfile.Upload() # Upload the file.


In [80]:
# #download files from google drive

# for i, file in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
# 	print('Downloading {} file from GDrive ({}/{})'.format(file['title'], i, len(file_list)))
# 	file.GetContentFile(file['title'])

In [81]:
# # Create a GoogleDriveFile instance with title 'test.txt'.
# file1 = drive.CreateFile({'parents': [{'id': '1cIMiqUDUNldxO6Nl-KVuS9SV-cWi9WLi'}],'title': 'test.txt'})  
# # Set content of the file from the given string.
# file1.SetContentString('Hello World!') 
# file1.Upload()

# # see files in google drive
# file2 = drive.CreateFile({'id': file1['id']})
# file2.GetContentString('test.txt')

In [82]:
# Separate Target Variable and Predictor Variables

TargetVariable='MEDV'
Predictors=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

X=clean_data[Predictors].values
y=clean_data[TargetVariable].values

# Selecting the final set of predictors for the deployment


### Sandardization of data ###
from sklearn.preprocessing import MinMaxScaler


# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)

# Generating the standardized values of X
X=PredictorScalerFit.transform(X)

print(X.shape)
print(y.shape)

(490, 13)
(490,)


In [83]:


# Saving the Python objects as serialized files can be done using pickle library
# Here let us save the Final model
with open('Final_Model.pkl', 'wb') as fileWriteStream:
    pickle.dump(all_data_model, fileWriteStream)
    # Don't forget to close the filestream!
    fileWriteStream.close()
    
print('pickle file of Predictive Model is saved at Location:',os.getcwd())

pickle file of Predictive Model is saved at Location: /Users/ioanahritcu/Documents/Projects/boston housing


In [84]:
def FunctionPredictResult(InputData):
    import pandas as pd
    Num_Inputs=InputData.shape[0]
    
    # Making sure the input data has same columns as it was used for training the model
    # Also, if standardization/normalization was done, then same must be done for new input
    
    # Appending the new data with the Training data
    DataForML=pd.read_pickle('DataForML.pkl')
    InputData=InputData.append(DataForML)

            
    # Maintaining the same order of columns as it was during the model training
    Predictors=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT',]
    
    # Generating the input values to the model
    X=InputData[Predictors].values[0:Num_Inputs]
    
    # Generating the standardized values of X since it was done while model training also
    X=PredictorScalerFit.transform(X)
    
    # Loading the Function from pickle file
    import pickle
    with open('Final_Model.pkl', 'rb') as fileReadStream:
        PredictionModel=pickle.load(fileReadStream)
        # Don't forget to close the filestream!
        fileReadStream.close()
            
    # Genrating Predictions
    Prediction=PredictionModel.predict(X)
    PredictionResult=pd.DataFrame(Prediction, columns=['Prediction'])
    return(PredictionResult)

In [85]:
NewSampleData=pd.DataFrame(
data=[[10,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311.0,15.2,392.52],
     [10,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311.0,15.2,392.52]],
columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT',])

print(NewSampleData)

# Calling the Function for prediction
FunctionPredictResult(InputData= NewSampleData)


   CRIM       ZN  INDUS  CHAS  NOX     RM    AGE   DIS     RAD  TAX  PTRATIO  \
0    10  0.22489   12.5  7.87    0  0.524  6.377  94.3  6.3467    5    311.0   
1    10  0.22489   12.5  7.87    0  0.524  6.377  94.3  6.3467    5    311.0   

      B   LSTAT  
0  15.2  392.52  
1  15.2  392.52  


/var/folders/p3/dhbmym6s0gb3_qp1wrvjs0rr0000gn/T/ipykernel_90267/1479496327.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  InputData=InputData.append(DataForML)


,Prediction
0,-474.647069
1,-474.647069


In [87]:
# Creating the function which can take inputs and return prediction
def FunctionGeneratePrediction(inp_LSTAT , inp_RM, inp_PTRATIO, inp_CRIM, inp_ZN, inp_INDUS, inp_CHAS, inp_NOX, inp_AGE, inp_DIS, inp_RAD, inp_TAX, inp_B):
    
    # Creating a data frame for the model input
    SampleInputData=pd.DataFrame(data=[[inp_CRIM, inp_ZN, inp_INDUS, inp_CHAS, inp_NOX, inp_AGE, inp_DIS, inp_RAD, inp_TAX, inp_B, inp_LSTAT , inp_RM, inp_PTRATIO]],
    columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])

    # Calling the function defined above using the input parameters
    Predictions=FunctionPredictResult(InputData= SampleInputData)
    return(Predictions)

    # Returning the predictions;
    return(Predictions.to_json())

# Function call
FunctionGeneratePrediction(inp_CRIM =0.5 ,inp_ZN=6.78, inp_INDUS = 6.56, inp_CHAS=5.6, inp_NOX=4.6, inp_AGE=7.9, inp_DIS = 544.3 , inp_RAD=21.56, inp_TAX = 43.56, inp_B = 5.6, inp_LSTAT=4.98,inp_RM=6.5,inp_PTRATIO=15.3)

/var/folders/p3/dhbmym6s0gb3_qp1wrvjs0rr0000gn/T/ipykernel_90267/1479496327.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  InputData=InputData.append(DataForML)


,Prediction
0,-30.27832
